In [3]:
from webapp import create_app
from vk_parser import *
import re
from numba import jit
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Обработка текста.

- Обработка текста через регулярные выражения.
- Удаление всех url из текста.
- Удаление хэштегов, если они есть в тексте.
- Удаление имён пользователей.
- Удаление пунктуации и цифр.
- Приведение всего текста к нижнему регистру.

In [4]:
def preprocess_text(text):
    text = text.lower().replace('ё', 'е')
    text = text.replace('d', '')
    text = text.replace('rt', '')
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text)
    text = re.sub('@[^\s]+', '', text)
    text = re.sub(r'[a-zA-Z]', '', text)
    text = re.sub(r'\b[a-zA-Zа-яА-Я1-9]\b', '', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip()

# Запрос к БД на извлечение текста комментарий.

In [5]:
app = create_app()
with app.app_context():
    query = Comment.query.all()
    preprop_comm = []

    for comment in query:
        preprop_comm.append(preprocess_text(comment.comment_text))

In [6]:
bd_data = [preprocess_text(t) for t in preprop_comm]

In [7]:
print(len(bd_data))

1220


In [12]:
bd_tokenized_commens = tokenaizer(bd_data)

In [154]:
print(len(bd_tokenized_commens))

1220


In [14]:
bd_prep_comments = []
stop_words = set(stopwords.words("russian"))
for words in bd_tokenized_commens:
    for word in words:
        if word not in stop_words:
            bd_prep_comments.append(word)   

In [15]:
len(bd_prep_comments)

7215

In [16]:
morph = pymorphy2.MorphAnalyzer()
default_list = []
for word in bd_prep_comments:
    lem_word = morph.parse(str(word))[0].normal_form
    default_list.append(lem_word)

In [18]:
len(default_list)

7215

# Считаем частоту слов в нашем предобработанном датасете

In [44]:
wordfreq = {}

for token in default_list:
    if token not in wordfreq.keys():
        wordfreq[token] = 1
    else:
        wordfreq[token] += 1

In [45]:
wordfreq

{'толсто': 2,
 'аж': 1,
 'тонко': 1,
 'весело': 2,
 'казаться': 6,
 'это': 133,
 'пародия': 2,
 'мойти': 1,
 'пайтон': 1,
 'типичный': 5,
 'осада': 5,
 'железный': 7,
 'войнов': 1,
 'крепость': 1,
 'имперский': 5,
 'кулак': 3,
 'просто': 15,
 'немного': 7,
 'унижение': 2,
 'плакса': 1,
 'удивлённый': 2,
 'кто': 14,
 'интересно': 13,
 'читать': 18,
 'классно': 1,
 'жв': 2,
 'описать': 3,
 'грэм': 1,
 'макнилла': 2,
 'омнибус': 2,
 'воин': 5,
 'полукровка': 1,
 'хонсю': 3,
 'хотеть': 18,
 'ааа': 1,
 'перевод': 9,
 'юмор': 2,
 'весь': 23,
 'полок': 1,
 'смеяться': 1,
 'ересь': 17,
 'госпожа': 3,
 'комиссар': 3,
 'бдеть': 1,
 'дисциплина': 1,
 'очень': 13,
 'практичный': 1,
 'шланг': 2,
 'голова': 11,
 'входить': 1,
 'дерьмо': 8,
 'вкачивать': 1,
 'выкачивать': 1,
 'какой': 18,
 'крыло': 5,
 'атасный': 1,
 'ребята': 7,
 'мм': 2,
 'лоял': 1,
 'ухо': 1,
 'положить': 2,
 'подача': 1,
 'варп': 9,
 'даста': 1,
 'друг': 11,
 'просить': 1,
 'фула': 1,
 'братец': 1,
 'грозить': 1,
 'инквизиция': 3

In [19]:
res = np.array(default_list)

In [20]:
unique_res = np.unique(res) 
unique_res

array(['ааа', 'ааааа', 'абаддон', ..., 'ярость', 'ясно', 'ёлочка'],
      dtype='<U19')

In [21]:
len(unique_res)

3507

In [22]:
bd_unique_words = unique_res.tolist()

# Предобработка текста.

- Используем корпус Юлии Рубцовой русских твитов для тренировки модели.
- Скачиваем корпус позитивных и негативных твитов с сайта http://study.mokoron.com/
- Добавляем дополнительную колонку для позитивных (1) и негативных твитов (0), тем самым размечая данные.

In [23]:
import pandas as pd
import numpy as np

# Считываем данные
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv('training_data/positive.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])
data_negative = pd.read_csv('training_data/negative.csv', sep=';', error_bad_lines=False, names=n, usecols=['text'])

# Формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])

raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis=0)
labels = [1] * sample_size + [0] * sample_size

data = [preprocess_text(t) for t in raw_data]

In [24]:
print(len(labels), len(raw_data), len(data))

223846 223846 223846


- Скачиваем набор русских стоп-слов для предобработки наших твитов.

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kamikaze666\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

- Разбиваем предобработанные слова на токены.
- Удаляем стоп слова.
- Производим лемматизацию или стемминг текста.
- Удаляем короткие слова длинной меньше 2 символов.

In [19]:
%timeit tokenized_words = [word_tokenize(sent) for sent in data]

26.1 s ± 2.21 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
@jit
def tokenaizer(data):
    tokens= [word_tokenize(sent) for sent in data]
    return tokens

In [29]:
tokenized_words = tokenaizer(data)

In [38]:
print(len(tokenized_words))

223846


# Стемминг слов

In [41]:
snowball = SnowballStemmer(language="russian")
def stemmer(word, stem_init):
    return stem_init.stem(str(word))

In [42]:
stop_words = set(stopwords.words("russian"))
def delete_stop_words_with_stemming(words, stop_words):    
    default_list = []
    for word in words:
        if word not in stop_words:
            stemmed_word = stemmer(word, snowball)
            default_list.append(stemmed_word)

    return list(default_list)

In [43]:
stemmed_words = []
counter = 0
for part_of_list in tokenized_words:
    stemmed_words.append(delete_stop_words_with_stemming(part_of_list, stop_words))
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 7
Processed 2000 of 6
Processed 3000 of 15
Processed 4000 of 9
Processed 5000 of 14
Processed 6000 of 14
Processed 7000 of 7
Processed 8000 of 16
Processed 9000 of 19
Processed 10000 of 11
Processed 11000 of 7
Processed 12000 of 6
Processed 13000 of 9
Processed 14000 of 8
Processed 15000 of 13
Processed 16000 of 8
Processed 17000 of 6
Processed 18000 of 11
Processed 19000 of 9
Processed 20000 of 10
Processed 21000 of 11
Processed 22000 of 2
Processed 23000 of 7
Processed 24000 of 12
Processed 25000 of 7
Processed 26000 of 11
Processed 27000 of 10
Processed 28000 of 5
Processed 29000 of 13
Processed 30000 of 11
Processed 31000 of 16
Processed 32000 of 3
Processed 33000 of 5
Processed 34000 of 7
Processed 35000 of 17
Processed 36000 of 14
Processed 37000 of 6
Processed 38000 of 9
Processed 39000 of 16
Processed 40000 of 5
Processed 41000 of 6
Processed 42000 of 7
Processed 43000 of 6
Processed 44000 of 7
Processed 45000 of 9
Processed 46000 of 13
Processed 47000 of 9
Pr

# Лемматизация слов.

"Экземпляры класса MorphAnalyzer обычно занимают порядка 15Мб оперативной памяти (т.к. загружают в память словари, данные для предсказателя и т.д.); старайтесь организовать свой код так, чтобы создавать экземпляр MorphAnalyzer заранее и работать с этим единственным экземпляром в дальнейшем."

https://pymorphy2.readthedocs.io/en/latest/user/guide.html

In [35]:
import pymorphy2

In [36]:
morph = pymorphy2.MorphAnalyzer()
def delete_stop_words_with_lemmatizing(words, morph):
    stop_words = set(stopwords.words("russian"))
    default_list = []
    for word in words:
        if word not in stop_words:
            lem_word = morph.parse(str(word))[0].normal_form
            default_list.append(lem_word)
    return list(default_list)

In [37]:
lemmatized_words = []
counter = 0
for part_of_list in tokenized_words:
    lemmatized_words.append(delete_stop_words_with_lemmatizing(part_of_list, morph))   
    counter += 1
    if counter%1000 == 0:
        print("Processed {} of {}".format(counter, len(part_of_list)))

Processed 1000 of 7
Processed 2000 of 6
Processed 3000 of 15
Processed 4000 of 9
Processed 5000 of 14
Processed 6000 of 14
Processed 7000 of 7
Processed 8000 of 16
Processed 9000 of 19
Processed 10000 of 11
Processed 11000 of 7
Processed 12000 of 6
Processed 13000 of 9
Processed 14000 of 8
Processed 15000 of 13
Processed 16000 of 8
Processed 17000 of 6
Processed 18000 of 11
Processed 19000 of 9
Processed 20000 of 10
Processed 21000 of 11
Processed 22000 of 2
Processed 23000 of 7
Processed 24000 of 12
Processed 25000 of 7
Processed 26000 of 11
Processed 27000 of 10
Processed 28000 of 5
Processed 29000 of 13
Processed 30000 of 11
Processed 31000 of 16
Processed 32000 of 3
Processed 33000 of 5
Processed 34000 of 7
Processed 35000 of 17
Processed 36000 of 14
Processed 37000 of 6
Processed 38000 of 9
Processed 39000 of 16
Processed 40000 of 5
Processed 41000 of 6
Processed 42000 of 7
Processed 43000 of 6
Processed 44000 of 7
Processed 45000 of 9
Processed 46000 of 13
Processed 47000 of 9
Pr

In [39]:
len(lemmatized_words)

223846

# Word embedding (Векторное представление слов)

- Сделаем векторизацию текста 4-мя способами BoW, TF-IDF, Word2Vec, doc2vec.
- На этих данных обучим модели и сравним результаты.

Subsampling